## Chatbot with tkinter

Here we gonna build a simple chatbot using *tkinter* package. Lets first load our model and other necessary files.

In [15]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
import json
import random

In [16]:
model = keras.models.load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))
lencoder = pickle.load(open('lencoder.pkl', 'rb'))

First we have to define a function which will be used to predict the tag for our sentence.

In [17]:
def predict_class(sentence, vectorizer, lencoder, model):
    yhat = model.predict(vectorizer.transform([sentence]).todense())
    tag = lencoder.inverse_transform([np.argmax(yhat)])[0]
    pred = {'tag': tag, 'prob': np.round(np.max(yhat), 2)}
    return pred

We will also need a function which can randomly pick up the response of the chatbot based on the predicted tag.

In [18]:
def chatbot_response(sentence, vectorizer, lencoder, model, intents):
    pred = predict_class(sentence, vectorizer, lencoder, model)
    for i in intents['intents']:
        if i['tag'] == pred['tag']:
            result = random.choice(i['responses'])
            break
    return result

Now we can build a simple chatbot. It will consist of the *Send* button which will call a *send* function after user enters any sentence in the text field. This text will be read and preprocessed in the *predict_class* function which will also predict the tag. Based on this tag *chatbot_response* function will choose a random response and print it in the text field.

In [19]:
import tkinter
from tkinter import *

In [20]:
def send():
    msg = EntryBox.get('1.0', 'end-1c').strip()
    EntryBox.delete('0.0', END)
    
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, 'You: ' + msg + '\n\n')
        ChatLog.config(foreground='#442265', font=('Verdana, 12'))
        res = chatbot_response(msg, vectorizer, lencoder, model, intents)
        ChatLog.insert(END, 'Bot: ' + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

In [21]:
base = Tk()
base.title('Hello')
base.geometry('400x500')
base.resizable(width=FALSE, height=FALSE)
ChatLog = Text(base, bd=0, bg='white', height='8', width='50', font='Arial')
ChatLog.config(state=DISABLED)
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor='heart')
ChatLog['yscrollcommand'] = scrollbar.set
SendButton = Button(base, font=('Verdana',12,'bold'), text='Send',
                    width='12', height='5', bd=0, bg='#32de97',
                    activebackground='#3c9d9b', fg='#ffffff', command=send)
EntryBox = Text(base, bd=0, bg='white', width='29', height='5', font='Arial')
scrollbar.place(x=376,y=6,height=386)
ChatLog.place(x=6,y=6,height=386,width=370)
EntryBox.place(x=128,y=401,height=90,width=265)
SendButton.place(x=6,y=401,height=90)

In [22]:
base.mainloop()